In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [3]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

In [4]:
loader = TextLoader("./data/state_of_the_union.txt")
loaded_document = loader.load()

In [5]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
chunks_of_text = text_splitter.split_documents(loaded_document)

In [6]:
len(chunks_of_text)

42

In [7]:
embeddings = OpenAIEmbeddings()

vector_db = FAISS.from_documents(chunks_of_text, embeddings)

In [8]:
vector_db

In [9]:
retriever = vector_db.as_retriever()

In [10]:
response = retriever.invoke("what did he say about ketanji brown jackson?")

In [11]:
response

[Document(metadata={'source': './data/state_of_the_union.txt'}, page_content='Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.'),
 Document(metadata={'source': './data/state_of_the_union.txt'}, page_content='A former top litigator in private practice. A f

In [12]:
retriever = vector_db.as_retriever(search_kwargs={"k": 3})

In [13]:
response = retriever.invoke("what did he say about ketanji brown jackson?")

In [14]:
response

[Document(metadata={'source': './data/state_of_the_union.txt'}, page_content='Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.'),
 Document(metadata={'source': './data/state_of_the_union.txt'}, page_content='A former top litigator in private practice. A f

# Simple use with LCEL (Langchain Expression Language)

In [15]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [18]:
template = """Answer the question based only on the following context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI(model="gpt-4.1-mini")

In [22]:
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])


chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()} # Parrallel runable giu nguyen input
    | prompt
    | model
    | StrOutputParser()
)

In [20]:
response = chain.invoke("what did he say about ketanji brown jackson?")

In [21]:
response

'He said that Ketanji Brown Jackson is one of the nation’s top legal minds who will continue Justice Breyer’s legacy of excellence. He described her as a former top litigator in private practice, a former federal public defender, and someone from a family of public school educators and police officers. He also called her a consensus builder and noted that since her nomination, she has received broad support—from the Fraternal Order of Police to former judges appointed by both Democrats and Republicans.'

In [23]:
type(response)

str